# To Loan or not to Loan - Data Mining Problem

In [1]:
if (!require("dplyr")) install.packages("dplyr")
if (!require("corrplot")) install.packages("corrplot")


Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: corrplot

corrplot 0.92 loaded



## Load datasets

We start by reading the data from the csv files that are available and uniformizing all the values such as "NaN", "?", "" or " " to be NA, which makes working with them easier.

In [2]:
# Read csv files
account_data <- read.csv("../data/account.csv", sep = ";")
card_data <- read.csv("../data/card_dev.csv", sep = ";")
client_data <- read.csv("../data/client.csv", sep = ";")
disp_data <- read.csv("../data/disp.csv", sep = ";")
district_data <- read.csv("../data/district.csv",
  sep = ";", na.strings = c("NaN", "?"))
loan_data <- read.csv("../data/loan_dev.csv", sep = ";")
trans_data <- read.csv("../data/trans_dev.csv", sep = ";")

# Change empty values to NA
account_data <-
  replace(account_data, (account_data == "" | account_data == " "), NA)
card_data <- replace(card_data, (card_data == "" | card_data == " "), NA)
client_data <-
  replace(client_data, (client_data == "" | client_data == " "), NA)
disp_data <-
  replace(disp_data, (disp_data == "" | disp_data == " "), NA)
district_data <-
  replace(district_data, (district_data == "" | district_data == " "), NA)
loan_data <- replace(loan_data, (loan_data == "" | loan_data == " "), NA)
trans_data <- replace(trans_data, (trans_data == "" | trans_data == " "), NA)

## Account Data

On the account dataset, we decided to parse the date from the YYMMDD format to YYYY-MM-DD, and then use it to calculate a new attribute called "age_days", which stores the age of the account in days. This will be useful to be able to compare statistics from accounts with different ages or other derived attributes.

In [3]:
account_data <- transform(account_data, acc_creation_date = as.Date(
  paste(
    paste("19", date %/% 10000, sep = ""),
    (date %/% 100) %% 100,
    date %% 100,
    sep = "-"
  ),
  format = "%Y-%m-%d"
))

account_data$age_days <- trunc(as.numeric(
  difftime(Sys.Date(), account_data$acc_creation_date, units = "days")
))

In [4]:
head(account_data)

,account_id,district_id,frequency,date,acc_creation_date,age_days
,<int>,<int>,<chr>,<int>,<date>,<dbl>
1,576,55,monthly issuance,930101,1993-01-01,10921
2,3818,74,monthly issuance,930101,1993-01-01,10921
3,704,55,monthly issuance,930101,1993-01-01,10921
4,2378,16,monthly issuance,930101,1993-01-01,10921
5,2632,24,monthly issuance,930102,1993-01-02,10920
6,1972,77,monthly issuance,930102,1993-01-02,10920


## Card data

For the card dataset, we only changed the data format and got rid of the day and month.

In [5]:
card_data <- transform(card_data, issued = format(as.Date(
  paste(
    paste("19", issued %/% 10000, sep = ""),
    (issued %/% 100) %% 100,
    issued %% 100,
    sep = "-"
  ),
  format = "%Y-%m-%d"
), "%Y"))

In [6]:
head(card_data)

,card_id,disp_id,type,issued
,<int>,<int>,<chr>,<chr>
1,1005,9285,classic,1993
2,104,588,classic,1994
3,747,4915,classic,1994
4,70,439,classic,1994
5,577,3687,classic,1994
6,377,2429,classic,1994


## Client data

For the client dataset, we transformed the "birthnumber" into gender and birthdate, which we later used to calculate the client's age (as of the last transaction's date). We also dropped the client district, as we will be considering the account district only.

In [7]:
client_data <- transform(client_data,
  gender = ifelse(((birth_number %/% 100) %% 100) <= 12, "M", "F")
)

client_data <- transform(client_data, birthday = as.Date(
  paste(
    paste("19", birth_number %/% 10000, sep = ""),
    ifelse(((birth_number %/% 100) %% 100) <= 12,
      (birth_number %/% 100) %% 100,
      ((birth_number %/% 100) %% 100) - 50
    ),
    birth_number %% 100,
    sep = "-"
  ),
  format = "%Y-%m-%d"
))

client_data <- subset(client_data, select = -c(birth_number, district_id))

We can see that the bank's data has a balanced gender distribution.

## District data

The district dataset was one of the ones that needed more work done. For this one, we renamed some columns for concistency, filled the empty values with the column average where possible, and transformed the data from 1995 and 1996 into an average of both and an attribute that tells whether or not those numbers grew from an year to the other. We then dropped some of the columns that were no longer needed.

In [8]:
# Rename code to district id to ease joins
# Rename columns for consistency
colnames(district_data)[colnames(district_data) == "code"] <- "district_id"
colnames(district_data)[
  colnames(district_data) == "average.salary"
] <- "average_salary"
colnames(district_data)[
  colnames(district_data) == "no..of.enterpreneurs.per.1000.inhabitants"
] <- "entrepreneur_rate"
colnames(district_data)[
  colnames(district_data) == "no..of.inhabitants"
] <- "population"
colnames(district_data)[
  colnames(district_data) == "ratio.of.urban.inhabitants"
] <- "urban_ratio"

# Fix values that were "?" to be the column average
district_data$unemploymant.rate..95[
  is.na(district_data$unemploymant.rate..95)
] <- mean(as.numeric(district_data$unemploymant.rate..95), na.rm = TRUE)
district_data$unemploymant.rate..96[
  is.na(district_data$unemploymant.rate..96)
] <- mean(as.numeric(district_data$unemploymant.rate..96), na.rm = TRUE)

district_data$no..of.commited.crimes..95[
  is.na(district_data$no..of.commited.crimes..95)
] <- mean(as.numeric(district_data$no..of.commited.crimes..95), na.rm = TRUE)
district_data$no..of.commited.crimes..96[
  is.na(district_data$no..of.commited.crimes..96)
] <- mean(as.numeric(district_data$no..of.commited.crimes..96), na.rm = TRUE)

# Calculate average between 95 and 96
district_data <- transform(district_data, unemployment_rate_avg =
  as.numeric(district_data$unemploymant.rate..95)
  + as.numeric(district_data$unemploymant.rate..96) / 2
)
district_data <- transform(district_data, crimes_rate_per_thousand =
  (as.numeric(district_data$unemploymant.rate..95)
  + as.numeric(district_data$unemploymant.rate..96) / 2)
  / as.numeric(district_data$population) 
  * 1000
)

# Calculate whether or not the unemployment/crimes has been growing
district_data <- transform(district_data, unemployment_growing =
  ifelse(
    as.numeric(district_data$unemploymant.rate..96) >
    as.numeric(district_data$unemploymant.rate..95),
    1,
    0
  )
)
district_data <- transform(district_data, crimes_growing =
  ifelse(
    as.numeric(district_data$no..of.commited.crimes..96) >
    as.numeric(district_data$no..of.commited.crimes..95),
    1,
    0
  )
)


district_data <- subset(district_data, select = -c(unemploymant.rate..95,
  unemploymant.rate..96, no..of.commited.crimes..95, no..of.commited.crimes..96,
  no..of.municipalities.with.inhabitants...499,
  no..of.municipalities.with.inhabitants.500.1999,
  no..of.municipalities.with.inhabitants.2000.9999,
  no..of.municipalities.with.inhabitants..10000,
  region,
  no..of.cities
))

In [9]:
head(district_data)

,district_id,name,population,urban_ratio,average_salary,entrepreneur_rate,unemployment_rate_avg,crimes_rate_per_thousand,unemployment_growing,crimes_growing
,<int>,<chr>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,Hl.m. Praha,1204953,100.0,12541,167,0.505,0.0004191035,1,1
2,2,Benesov,88884,46.7,8507,132,2.595,0.0291953557,1,1
3,3,Beroun,75232,41.7,8980,111,3.055,0.0406077201,1,0
4,4,Kladno,149893,67.4,9753,109,7.165,0.0478007645,1,1
5,5,Kolin,95616,51.4,9307,118,6.065,0.0634308066,1,1
6,6,Kutna Hora,77963,51.5,8546,126,4.960,0.0636199223,1,1


Like this, each district has a simpler representation, that is easier to use for comparisons between districts, as it uses statistics per thousand people instead of absolute counts.

From the previous graph, we gather that Most has the highest unemployment, at about 12%, and that there is no clear correlation between higher unemployment and higher crime rate

## Loan data

For the loan data, we only needed to parse the date to YYYY-MM-DD
Since this table contains the target variable, we can alreay try to find some correlations.

In [10]:
# Make date more readable
loan_data <- transform(loan_data, date = as.Date(
  paste(
    paste("19", date %/% 10000, sep = ""),
    (date %/% 100) %% 100,
    date %% 100,
    sep = "-"
  ),
  format = "%Y-%m-%d"
))

In [11]:
head(loan_data)

,loan_id,account_id,date,amount,duration,payments,status
,<int>,<int>,<date>,<int>,<int>,<int>,<int>
1,5314,1787,1993-07-05,96396,12,8033,-1
2,5316,1801,1993-07-11,165960,36,4610,1
3,6863,9188,1993-07-28,127080,60,2118,1
4,5325,1843,1993-08-03,105804,36,2939,1
5,7240,11013,1993-09-06,274740,60,4579,1
6,6687,8261,1993-09-13,87840,24,3660,1


We verified that the number of successfull loans is much higher than the number of unssuccessful loans, which we noted and decided to later use SMOTE techniques in order to better balance the dataset.

With this analysis, we can see that in the higher amount bins, the success rate seems to be lower, which means that it more likely for a bigger loan to fail.

As we can see, there doesn't seem to be a correlation between the duration or the amount paid per month and the success of a loan.

# Transaction data

In [12]:
head(trans_data)

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
,<int>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<int>
1,1548749,5270,930113,credit,credit in cash,800,800,NA,NA,NA
2,1548750,5270,930114,credit,collection from another bank,44749,45549,NA,IJ,80269753
3,3393738,11265,930114,credit,credit in cash,1000,1000,NA,NA,NA
4,3122924,10364,930117,credit,credit in cash,1100,1100,NA,NA,NA
5,1121963,3834,930119,credit,credit in cash,700,700,NA,NA,NA
6,2809952,9307,930124,credit,credit in cash,900,900,NA,NA,NA


As we can see in the table above, this table has a lot of NA values, which we will have to deal with.
For now we start with some simple transformations, such as renaming some columns, cleaning the "account" attribute (which could otherwise cause problems with the competition dataset), parsing the date and dropping the "type" column, as that information can be conveied by adding a signal to the amount and getting information from the k_symbol and type columns.

In [13]:
# Rename k_symbol to category
colnames(trans_data)[colnames(trans_data) == "k_symbol"] <- "category"

# Account id of 0 does not exist, so we replace it by NA (competition)
trans_data$account <- replace(trans_data$account, (trans_data$account == 0), NA)

# Parse the date
trans_data <- transform(trans_data, date = as.Date(
  paste(
    paste("19", date %/% 10000, sep = ""),
    (date %/% 100) %% 100,
    date %% 100,
    sep = "-"
  ),
  format = "%Y-%m-%d"
))

# Make amount reflect if money entered or left the account
trans_data <- transform(trans_data,
  amount = ifelse(type == "credit", amount, -1 * amount)
)

In [14]:
head(trans_data)

,trans_id,account_id,date,type,operation,amount,balance,category,bank,account
,<int>,<int>,<date>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<int>
1,1548749,5270,1993-01-13,credit,credit in cash,800,800,NA,NA,NA
2,1548750,5270,1993-01-14,credit,collection from another bank,44749,45549,NA,IJ,80269753
3,3393738,11265,1993-01-14,credit,credit in cash,1000,1000,NA,NA,NA
4,3122924,10364,1993-01-17,credit,credit in cash,1100,1100,NA,NA,NA
5,1121963,3834,1993-01-19,credit,credit in cash,700,700,NA,NA,NA
6,2809952,9307,1993-01-24,credit,credit in cash,900,900,NA,NA,NA


Now, we have to deal with the NA values.

In [15]:
colSums(is.na(trans_data)) # count number of NA values for each column

trans_id account_id       date       type  operation     amount    balance 
         0          0          0          0      70761          0          0 
  category       bank    account 
    204309     299443     299443

The most problematic columns are "operation", "category", "bank" and "account". We can now try to understand under which circunstances these are NA and what we can do to fix them.

In [16]:
head(trans_data[is.na(trans_data$operation),], 10)

,trans_id,account_id,date,type,operation,amount,balance,category,bank,account
,<int>,<int>,<date>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<int>
11,3453599,6473,1993-01-31,credit,NA,17.0,21078.0,interest credited,NA,NA
13,3669814,5270,1993-01-31,credit,NA,110.2,45659.2,interest credited,NA,NA
14,3479816,9307,1993-01-31,credit,NA,0.8,6182.8,interest credited,NA,NA
47,3673340,5891,1993-02-28,credit,NA,21.6,29015.6,interest credited,NA,NA
48,3453600,6473,1993-02-28,credit,NA,102.0,36270.0,interest credited,NA,NA
49,3509732,9265,1993-02-28,credit,NA,21.3,28707.9,interest credited,NA,NA
50,3520588,10364,1993-02-28,credit,NA,38.3,19831.3,interest credited,NA,NA
51,3657066,3834,1993-02-28,credit,NA,34.7,21209.7,interest credited,NA,NA
52,3589600,1801,1993-02-28,credit,NA,2.9,702.9,interest credited,NA,NA


Looking at this table, all the values seem to have something in common: the category is "interest credited". Let's test this hypotesis by checking what are the possible values for the category when the operation is NA.

In [17]:
unique(trans_data[is.na(trans_data$operation),]$category)

[1] "interest credited"

So, now that we confirmed that our hypotesis was right, we can create a new value for operation and call it "interest credited". 

In [18]:
trans_data <- transform(trans_data,
  operation = ifelse(
    is.na(trans_data$operation) | trans_data$operation == "" |
      trans_data$operation == " ",
    "interest credited",
    operation
  )
)

In [19]:
sum(is.na(trans_data$operation))

[1] 0

As a result, we no longer have NA values for the operation attribute.

Now we try to do the same analysis for the category.

In [20]:
head(trans_data[is.na(trans_data$category),])

,trans_id,account_id,date,type,operation,amount,balance,category,bank,account
,<int>,<int>,<date>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<int>
1,1548749,5270,1993-01-13,credit,credit in cash,800,800,NA,NA,NA
2,1548750,5270,1993-01-14,credit,collection from another bank,44749,45549,NA,IJ,80269753
3,3393738,11265,1993-01-14,credit,credit in cash,1000,1000,NA,NA,NA
4,3122924,10364,1993-01-17,credit,credit in cash,1100,1100,NA,NA,NA
5,1121963,3834,1993-01-19,credit,credit in cash,700,700,NA,NA,NA
6,2809952,9307,1993-01-24,credit,credit in cash,900,900,NA,NA,NA


In [21]:
unique(trans_data[is.na(trans_data$category), ]$operation)

[1] "credit in cash"               "collection from another bank"
[3] "withdrawal in cash"           "remittance to another bank"  
[5] "credit card withdrawal"

This time we were not so lucky, which means we can't do the same thing as before. As such, we decided to mark those as "other".

In [22]:
trans_data$category[is.na(trans_data$category)] <- "other"

We then drop the columns that we won't need

In [23]:
trans_data <- subset(trans_data, select = -c(type))

Now that we did some basic processing of every table, the "bank" and "account" will be dropped as we drop every column with more than 70% of null values.

In [24]:
## Remove columns with more than 70% NA
account_data <- account_data %>% select(where(~ mean(is.na(.)) < 0.7))
card_data <- card_data %>% select(where(~ mean(is.na(.)) < 0.7))
client_data <- client_data %>% select(where(~ mean(is.na(.)) < 0.7))
district_data <- district_data %>% select(where(~ mean(is.na(.)) < 0.7))
disp_data <- disp_data %>% select(where(~ mean(is.na(.)) < 0.7))
loan_data <- loan_data %>% select(where(~ mean(is.na(.)) < 0.7))
trans_data <- trans_data %>% select(where(~ mean(is.na(.)) < 0.7))

In [25]:
colSums(is.na(account_data))
colSums(is.na(card_data))
colSums(is.na(district_data))
colSums(is.na(disp_data))
colSums(is.na(loan_data))
colSums(is.na(trans_data))

account_id       district_id         frequency              date 
                0                 0                 0                 0 
acc_creation_date          age_days 
                0                 0

card_id disp_id    type  issued 
      0       0       0       0

district_id                     name               population 
                       0                        0                        0 
             urban_ratio           average_salary        entrepreneur_rate 
                       0                        0                        0 
   unemployment_rate_avg crimes_rate_per_thousand     unemployment_growing 
                       0                        0                        0 
          crimes_growing 
                       0

disp_id  client_id account_id       type 
         0          0          0          0

loan_id account_id       date     amount   duration   payments     status 
         0          0          0          0          0          0          0

trans_id account_id       date  operation     amount    balance   category 
         0          0          0          0          0          0          0

Now, there is no table with NA values.

## Aggregate the transactions data

The transactions dataframe is the biggest one, and in order to relate it to the loan table, where our target variable is, we need to aggregate it by account_id.

In [26]:
# Aggregate transactions data
aggregated_trans <- trans_data %>%
  # Group by account
  group_by(account_id) %>%
  arrange(date, .by_group = TRUE) %>%
  # Add number of transactions per account
  mutate(trans_count = n()) %>%
  # Count credits/withdrawals
  mutate(credit_count = sum(amount >= 0)) %>%
  mutate(credit_ratio = mean(amount >= 0)) %>%
  mutate(withdrawal_count = sum(amount < 0)) %>%
  mutate(withdrawal_ratio = mean(amount < 0)) %>%
  # Amount stats
  mutate(smallest_transaction = amount[which.min(abs(amount))][1]) %>%
  mutate(biggest_transaction = amount[which.max(abs(amount))][1]) %>%
  mutate(transactions_net = sum(amount)) %>%
  # Balance stats
  mutate(balance_min = min(balance)) %>%
  mutate(balance_max = max(balance)) %>%
  mutate(current_balance = last(balance)) %>%
  mutate(times_negative_balance = sum(balance < 0)) %>%
  # Operation ratios
  mutate(credit_cash_ratio =
    mean(as.character(operation) == "credit in cash")) %>%
  mutate(collection_bank_ratio =
    mean(as.character(operation) == "collection from another bank")) %>%
  mutate(interest_ratio =
    mean(as.character(operation) == "interest credited")) %>%
  mutate(withdrawal_cash_ratio =
    mean(as.character(operation) == "withdrawal in cash")) %>%
  mutate(remittance_bank_ratio =
    mean(as.character(operation) == "remittance to another bank")) %>%
  mutate(withdrawal_card_ratio =
    mean(as.character(operation) == "credit card withdrawal")) %>%
  mutate(sanctions =
    sum(as.character(category) == "sanction interest if negative balance")) %>%

  distinct()


trans_agg <- subset(aggregated_trans, select =
  -c(trans_id, operation, amount, balance, date, category)
)


In [27]:
head(aggregated_trans)

trans_id,account_id,date,operation,amount,balance,category,trans_count,credit_count,credit_ratio,⋯,balance_max,current_balance,times_negative_balance,credit_cash_ratio,collection_bank_ratio,interest_ratio,withdrawal_cash_ratio,remittance_bank_ratio,withdrawal_card_ratio,sanctions
<int>,<int>,<date>,<chr>,<dbl>,<dbl>,<chr>,<int>,<int>,<dbl>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,1,1995-03-24,credit in cash,1000.0,1000.0,other,113,50,0.4424779,⋯,30414.8,12674.5,0,0.07079646,0.1858407,0.1858407,0.4159292,0.1415929,0,0
5,1,1995-04-13,collection from another bank,3679.0,4679.0,other,113,50,0.4424779,⋯,30414.8,12674.5,0,0.07079646,0.1858407,0.1858407,0.4159292,0.1415929,0,0
199,1,1995-04-23,credit in cash,12600.0,17279.0,other,113,50,0.4424779,⋯,30414.8,12674.5,0,0.07079646,0.1858407,0.1858407,0.4159292,0.1415929,0,0
3530438,1,1995-04-30,interest credited,19.2,17298.2,interest credited,113,50,0.4424779,⋯,30414.8,12674.5,0,0.07079646,0.1858407,0.1858407,0.4159292,0.1415929,0,0
6,1,1995-05-13,collection from another bank,3679.0,20977.2,other,113,50,0.4424779,⋯,30414.8,12674.5,0,0.07079646,0.1858407,0.1858407,0.4159292,0.1415929,0,0
200,1,1995-05-23,credit in cash,2100.0,23077.2,other,113,50,0.4424779,⋯,30414.8,12674.5,0,0.07079646,0.1858407,0.1858407,0.4159292,0.1415929,0,0


With this, we created some new variables:

- **trans_count**: number of transactions for the account;
- **credit_count**: number of positive transactions for the account;
- **credit_ratio**: ratio of positive transactions for the account;
- **withdrawal_count**: number of negative transactions for the account;
- **withdrawal_ratio**: ratio of negative transactions for the account;
- **smallest_transaction**: smallest transaction for the account;
- **biggest_transaction**: biggest transaction for the account;
- **transactions_net**: difference of all money that entered and left the account;
- **balance_min**: lowest balance the account has had;
- **balance_max**: highest balance the account has had;
- **current_balance**: current balance;
- **times_negative_balance**: number of times the account's balance has been negative;
- **credit_cash_ratio**: ratio of "credit in cash" operations;
- **collection_bank_ratio**: ratio of "collection from another bank" operations;
- **interest_ratio**: ratio of "interest credited" operations;
- **withdrawal_cash_ratio**: ratio of "withdrawal in cash" operations;
- **remittance_bank_ratio**: ratio of "remittance to another bank" operations;
- **withdrawal_card_ratio**: ratio of "credit card withdrawal" operations;
- **sanctions**: number of "sanction interest if negative balance" operations;

And dropped the variables that were now redundant: trans_id, operation, amount, balance, date and category.

## Merging the data

Now that the data is ready, we will join every table and derive new attributes.

In [28]:
last_transaction <- max(trans_data$date) # need to calculate age

data <- loan_data %>%
  rename(loan_date = date) %>%
  left_join(account_data, by = "account_id") %>%
  left_join(trans_agg, by = "account_id") %>%
  mutate(transactions_net = transactions_net / age_days) %>%
  mutate(sanctions_rate = sanctions / age_days) %>%
  rename(daily_transactions_net = transactions_net) %>%
  left_join(disp_data, by = "account_id") %>%
  filter(type == "OWNER") %>%
  select(-age_days, -type, -sanctions) %>%
  left_join(card_data, "disp_id") %>%
  mutate(has_card = ifelse(!is.na(card_id), 1, 0)) %>%
  mutate(is_gold = ifelse((!is.na(type) & type == "gold"), 1, 0)) %>%
  select(-card_id, -type, -issued) %>%
  left_join(client_data, by = "client_id") %>%
  mutate(client_age = trunc(as.numeric(
  difftime(as.Date(last_transaction), birthday, units = "weeks")
) / 52.25)) %>%
  left_join(district_data, by = "district_id") %>%
  select(-c(name)) %>%
  mutate(can_afford_loan = ifelse(average_salary > payments, 1, 0)) %>%
  mutate(can_pay_until = current_balance / payments) %>%
  mutate(acc_age_when_loan = trunc(as.numeric(
    difftime(loan_date, acc_creation_date, units = "days")))
  ) %>%
  select(-c(acc_creation_date, account_id, district_id, date,
    disp_id, client_id
  )) %>%

  distinct()

In [29]:
head(data)

,loan_id,loan_date,amount,duration,payments,status,frequency,trans_count,credit_count,credit_ratio,⋯,urban_ratio,average_salary,entrepreneur_rate,unemployment_rate_avg,crimes_rate_per_thousand,unemployment_growing,crimes_growing,can_afford_loan,can_pay_until,acc_age_when_loan
,<int>,<date>,<int>,<int>,<int>,<int>,<chr>,<int>,<int>,<dbl>,⋯,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,5314,1993-07-05,96396,12,8033,-1,weekly issuance,4,4,1.0000000,⋯,81.8,9650,100,5.215,0.0550035860,1,0,1,2.502179,105
2,5316,1993-07-11,165960,36,4610,1,monthly issuance,37,17,0.4594595,⋯,73.5,8369,117,2.945,0.0261292355,1,0,1,11.325141,148
3,6863,1993-07-28,127080,60,2118,1,monthly issuance,24,15,0.6250000,⋯,53.5,8390,132,3.725,0.0478072821,1,1,1,9.571671,170
4,5325,1993-08-03,105804,36,2939,1,monthly issuance,25,13,0.5200000,⋯,58.0,8754,137,5.985,0.0554834523,1,1,1,11.668152,185
5,7240,1993-09-06,274740,60,4579,1,weekly issuance,27,13,0.4814815,⋯,100.0,12541,167,0.505,0.0004191035,1,1,1,8.985128,204
6,6687,1993-09-13,87840,24,3660,1,monthly issuance,17,14,0.8235294,⋯,41.3,8598,123,4.400,0.0816008605,1,1,1,11.980273,124


This has also allowed us to derive new attributes and improve old ones, such as:

- **transactions_net**: is now the net amount of money transacted per day;
- **sanctions_rate**: number of sanctions per day;
- **has_card**: whether or not there is a credit card associated with the account;
- **is_gold**: whether or not the account has a gold credit card;
- **client_age**: the age of the client as of the last transaction's date;
- **can_afford_loan**: whether or not the client's district has an average salary that can cover the loan's monthly payments;
- **can_pay_until**: how many months can the client's balance cover the loans payments for;
- **acc_age_when_loan**: how old the account was when the loan was made;

We also dropped some columns such as: age_days, type, sanctions, card_id, issued, acc_creation_date, account_id, district_id, date, disp_id, client_id.

We can then save the result to a csv file that will be used to train/test the model.

In [30]:
write.csv(data,'../data/dataframe.csv', row.names = FALSE)